In [15]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1, self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size= x.size()[1:]
        print(size)
        num_features=1
        for s in size:
            num_features *=s
            print(num_features)
        return num_features
net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [26]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [33]:
input = Variable(torch.randn(1,1,32,32))
out = net(input)

print(input)

print(out)

torch.Size([16, 5, 5])
16
80
400
Variable containing:
(0 ,0 ,.,.) = 
 -0.1878 -0.4889  0.9610  ...  -0.5380 -2.8870 -0.2491
 -1.2447  0.6028 -0.8362  ...  -0.2726  0.6051 -0.7941
 -1.0182  0.3515 -1.3481  ...  -1.0889 -0.3017  1.8228
           ...             ⋱             ...          
  0.7201  1.2421 -0.4994  ...  -0.0245 -0.6102 -0.5926
  0.5698  0.9252  0.0249  ...  -0.6629  0.2614  0.7670
 -1.0550  1.0534  1.3507  ...  -1.2479  0.0841  0.1472
[torch.FloatTensor of size 1x1x32x32]

Variable containing:
 0.1100 -0.0145  0.0441  0.0639 -0.0291  0.0356  0.1399  0.0518  0.0735  0.0661
[torch.FloatTensor of size 1x10]



In [28]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [31]:
output = net(input)
target = Variable(torch.arange(1,11))
criterion = nn.MSELoss()
loss=criterion(output,target)
print(loss)

torch.Size([16, 5, 5])
16
80
400
Variable containing:
 38.2286
[torch.FloatTensor of size 1]



In [32]:
net.zero_grad()
loss.backward()
lr_rate=0.01
for f in net.parameters():
    f.data.sub_(lr_rate*f.grad.data)
    